## Migration from Text Completions API to the new Messages API

The text completions API for Claude is now in legacy mode and it is recommended to migrate to 
the new Messages API. The Messages API is backward compatible with current Claude Models on Bedrock and requires that a modified JSON body be passed to the invoke methods. An example of a text completion API style body and prompt is shown below:

prompt='Human:\n\n What is quantum mechanics? \n\n Assistant:\n\n'

body = json.dumps({
                "prompt": prompt,
                "temperature": 0.5,
                "top_p": 0.9,
                "max_tokens_to_sample": 512,
            })

The new body will have the following form with an updated messages element.

body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 512,
            "temperature": 0.5,
            "top_p": 0.9,
            "system": system_prompt,
            "messages": messages
        }  
    ) 
    
The messages element represents a list of JSONL objects that include the role and content for each turn in an input. For a single input, the messages element is shown below where the "Human" tag from the text Completion API is replaced by "user" in the new API:

messages=[{ "role":'user', "content":[{'type':'text','text': "What is quantum mechanics?"}]}]

Lets see how this works with some examples with Claude 2.1

### Initialize packages

In [34]:
%pip install --upgrade pip
%pip install boto3>=1.33.2 --force-reinstall --quiet
%pip install botocore>=1.33.2 --force-reinstall --quiet


Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.7.0 requires botocore<1.31.65,>=1.31.16, but you have botocore 1.34.54 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.7.0 requires botocore<1.31.65,>=1.31.16, but you have botocore 1.34.54 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


#### Restart the kernel with the updated packages that are installed through the dependencies above

In [2]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [36]:
import boto3
import json
import base64

bedrock_client = boto3.client('bedrock-runtime',region_name='us-west-2')


### Helper function to pass our models, messages, and inference parameters

In [37]:
def generate_message(bedrock_runtime, model_id, messages, max_tokens,top_p,temp):

    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "messages": messages,
            "temperature": temp,
            "top_p": top_p
        }  
    )  
    
    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())

    return response_body

### Single-Input

In [7]:
messages=[{ "role":'user', "content":[{'type':'text','text': "What is quantum mechanics?"}]}]

generate_message(bedrock_client, model_id = 'anthropic.claude-v2:1',messages=messages,max_tokens=512,temp=0.5,top_p=0.9)

{'id': 'compl_012ae1AjPa6JMBg5sno8qofA',
 'type': 'message',
 'role': 'assistant',
 'content': [{'type': 'text',
   'text': 'Quantum mechanics is a fundamental theory in physics that describes the behavior of matter and energy at the subatomic level. Some key points about quantum mechanics:\n\n- It explains phenomena that classical physics cannot, such as the wave-particle duality of matter and quantum entanglement. At the quantum scale, particles can behave like waves and vice versa.\n\n- It introduces probabilities and uncertainty. The position and momentum of quantum particles cannot be precisely known simultaneously. There is always a degree of uncertainty.\n\n- It relies heavily on advanced mathematics. Quantum mechanics uses techniques like linear algebra, differential equations, and Hilbert spaces. \n\n- It leads to many counter-intuitive conclusions. In the quantum world, a particle can be in multiple places at once and teleportation is possible. These run contrary to our every

### Multi-turn Input

In [8]:
messages=[{ "role":'user', "content":[{'type':'text','text': "What is quantum mechanics? "}]},\
         { "role":'assistant', "content":[{'type':'text','text': "It is a branch of physics that \
         describes how matter and energy interact with discrete energy values "}]},\
         { "role":'user', "content":[{'type':'text','text': "Can you explain a bit more about discrete energies?"}]}]

generate_message(bedrock_client, model_id = 'anthropic.claude-v2:1',messages=messages,max_tokens=512,temp=0.5,top_p=0.9)

{'id': 'compl_01S6TATWwu61oVGXoEUvRECN',
 'type': 'message',
 'role': 'assistant',
 'content': [{'type': 'text',
   'text': "Sure, the key idea around discrete energies in quantum mechanics is:\n\n- Matter and energy at the subatomic level exist only in discrete quantities that are multiples of a base quantity. For example, the energy levels of an electron orbiting an atom can only have certain specific values, not a continuum of values.\n\n- These discrete values are called quantum levels or states. An electron can jump between different discrete energy levels by emitting or absorbing a photon - a particle of light energy. \n\n- The photon energy exactly matches the difference in energy between the two electron levels according to the Planck relation: E=hf. Where h is Planck's constant and f is the photon frequency.\n\n- This quantization of energy levels explains phenomena like atomic emission spectra - when gases emit or absorb photons at very specific frequencies matching the energ

### Claude 3 with Text Completions API will give the following error: 

ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: "claude-3-sonnet-20240229" is not supported on this API. Please use the Messages API instead.

In [38]:
prompt='Human:\n\n What is quantum mechanics? \n\n Assistant:\n\n'

body = json.dumps({ "prompt": prompt, "temperature": 0.5, "top_p": 0.9, "max_tokens_to_sample": 512, })

response = bedrock_client.invoke_model(body=body, modelId="anthropic.claude-3-sonnet-20240229-v1:0")

ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: "claude-3-sonnet-20240229" is not supported on this API. Please use the Messages API instead.

### Claude 3-sonnet single-turn example with correct Messages API call

### Claude 3-sonnet multi-turn example with correct Messages API call

In [39]:
messages=[{ "role":'user', "content":[{'type':'text','text': "What is quantum mechanics? "}]},\
         { "role":'assistant', "content":[{'type':'text','text': "It is a branch of physics that \
         describes how matter and energy interact with discrete energy values "}]},\
         { "role":'user', "content":[{'type':'text','text': "Can you explain a bit more about discrete energies?"}]}]

generate_message(bedrock_client, model_id = "anthropic.claude-3-sonnet-20240229-v1:0",messages=messages,max_tokens=512,temp=0.5,top_p=0.9)

{'id': 'msg_01U3bFRtfwonjebg3d6YqnP6',
 'type': 'message',
 'role': 'assistant',
 'content': [{'type': 'text',
   'text': 'Sure, the concept of discrete energy values or levels is a key principle in quantum mechanics. It states that the energy of particles like electrons can only take on certain specific values, rather than varying continuously.\n\nSome key points about discrete energy levels:\n\n- Electrons bound to an atom can only exist in specific energy levels around the nucleus, they cannot have any arbitrary energy value.\n\n- These allowed energy levels are determined by the principles of quantum mechanics and are fixed for each atom/molecule.\n\n- When an electron absorbs or emits energy, it must jump between these fixed energy levels. It cannot change its energy by an arbitrary amount.\n\n- The discrete energy levels get further apart for higher energy states further from the nucleus.\n\n- Transitions between energy levels correspond to the absorption or emission of photons w